# Differences between LangGraph and AgentExecutor

Here we compare functionality between LangGraph's pre-built agent executors and LangChain's [AgentExecutor](https://api.python.langchain.com/en/latest/agents/langchain.agents.agent.AgentExecutor.html) class.

LangGraph supports multiple types of pre-built agent executors, including [`agent_executor`](https://github.com/langchain-ai/langgraph/blob/main/langgraph/prebuilt/agent_executor.py) and `chat_agent_executor`. `agent_executor`, like LangChain's AgentExecutor, is not on its own conversational; the agent iterates by updating actions and observations to an "agent scratchpad" in its prompt. `chat_agent_executor` is naturally conversational, and includes agent actions, tool results, and user messages in its prompt. We focus on LangGraph's `chat_agent_executor` for most cases.

## 0. Basic usage

First we define a model and a tool:

In [1]:
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI


model = ChatOpenAI()


@tool
def magic_function(input: int) -> int:
    """Applies a magic function to an input."""
    return input + 2


tools = [magic_function]


query = "what is the value of magic_function(3)?"

For AgentExecutor, we define a prompt with a placeholder for the agent's scratchpad. The agent can be invoked as follows:

In [2]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant"),
        ("human", "{input}"),
        MessagesPlaceholder("agent_scratchpad"),
    ]
)


agent = create_tool_calling_agent(model, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

agent_executor.invoke({"input": query})

{'input': 'what is the value of magic_function(3)?',
 'output': 'The value of `magic_function(3)` is 5.'}

LangGraph's `chat_agent_executor` manages a state that is defined by a list of messages. It will continue to process the list until there are no tool calls in the agent's output. To kick it off, we input a list of messages. The output will contain the entire state of the graph-- in this case, the conversation history.

In [3]:
from langgraph.prebuilt import chat_agent_executor

app = chat_agent_executor.create_tool_calling_executor(model, tools)


messages = app.invoke({"messages": [("human", query)]})
{
    "input": query,
    "output": messages["messages"][-1].content,
}

{'input': 'what is the value of magic_function(3)?',
 'output': 'The value of magic_function(3) is 5.'}

Note that we can easily continue the conversation by appending a new message to the list and invoking the agent again:

In [4]:
message_history = messages["messages"]

new_query = "Pardon?"

messages = app.invoke({"messages": message_history + [("human", new_query)]})
{
    "input": new_query,
    "output": messages["messages"][-1].content,
}

{'input': 'Pardon?',
 'output': 'I apologize for the confusion. The value of magic_function(3) is 5.'}

## 1. Visibility

### `verbose`

AgentExecutor supports a `verbose` flag that will log intermediate steps:

In [5]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

agent_executor.invoke({"input": query})



> Entering new AgentExecutor chain...

Invoking: `magic_function` with `{'input': 3}`


5The value of the magic_function(3) is 5.

> Finished chain.


{'input': 'what is the value of magic_function(3)?',
 'output': 'The value of the magic_function(3) is 5.'}

We can use LangGraph's streaming capabilities to provide similar visibility as toggling `AgentExecutor.verbose`:

In [6]:
def print_chunk(chunk: dict) -> None:
    if "agent" in chunk:
        message = chunk["agent"]["messages"][0]
        if message.tool_calls:
            print(f"Invoking: {message.tool_calls}")
        else:
            print(message.content)
    elif "action" in chunk:
        print(chunk["action"]["messages"][0].content)
    else:
        pass


for chunk in app.stream({"messages": [("human", query)]}):
    print_chunk(chunk)
    print("------")

Invoking: [{'name': 'magic_function', 'args': {'input': 3}, 'id': 'call_x0pZqrZtz0Gi73IYkTFTGdxd'}]
------
5
------
The value of the magic_function(3) is 5.
------


### `return_intermediate_steps`

Setting this parameter on AgentExecutor allows users to access `intermediate_steps`, which pairs agent actions (e.g., tool invocations) with their outcomes.

In [7]:
agent_executor = AgentExecutor(agent=agent, tools=tools, return_intermediate_steps=True)
result = agent_executor.invoke({"input": query})
print(result["intermediate_steps"])

[(ToolAgentAction(tool='magic_function', tool_input={'input': 3}, log="\nInvoking: `magic_function` with `{'input': 3}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_wHkRoBuMuX66giohKQkLZXO8', 'function': {'arguments': '{"input":3}', 'name': 'magic_function'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls'}, id='run-09e58bdb-0874-45cf-b6f4-0ecee91d1593', tool_calls=[{'name': 'magic_function', 'args': {'input': 3}, 'id': 'call_wHkRoBuMuX66giohKQkLZXO8'}], tool_call_chunks=[{'name': 'magic_function', 'args': '{"input":3}', 'id': 'call_wHkRoBuMuX66giohKQkLZXO8', 'index': 0}])], tool_call_id='call_wHkRoBuMuX66giohKQkLZXO8'), 5)]


By default LangGraph lets you access any key in the state. Because LangGraph's `agent_executor` includes `intermediate_steps` in the state, we can easily replicate:

In [8]:
from langgraph.prebuilt import create_agent_executor


agent_executor_app = create_agent_executor(agent, tools)

result = agent_executor_app.invoke({"input": query})
print(result["intermediate_steps"])

[(ToolAgentAction(tool='magic_function', tool_input={'input': 3}, log="\nInvoking: `magic_function` with `{'input': 3}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_IRViSR8rUJmo2tvJuiFZDpAA', 'function': {'arguments': '{"input":3}', 'name': 'magic_function'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 71, 'total_tokens': 85}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_d9767fc5b9', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-6bb2ee6a-367a-4bfd-b843-156db0149719-0', tool_calls=[{'name': 'magic_function', 'args': {'input': 3}, 'id': 'call_IRViSR8rUJmo2tvJuiFZDpAA'}])], tool_call_id='call_IRViSR8rUJmo2tvJuiFZDpAA'), '5')]


## 2. Limiting iterations

### `max_iterations`
`AgentExecutor` implements a `max_iterations` parameter, whereas this is controlled via `recursion_limit` in LangGraph.

Note that in AgentExecutor, an "iteration" includes a full turn of tool invocation and execution. In LangGraph, each step contributes to the recursion limit, so we will need to multiply by two (and add one) to get equivalent results.

If the recursion limit is reached, LangGraph raises a specific exception type, that we can catch and manage similarly to AgentExecutor.

In [9]:
@tool
def magic_function(input: str) -> str:
    """Applies a magic function to an input."""
    return "Sorry, there was an error. Please try again."


tools = [magic_function]

In [10]:
agent = create_tool_calling_agent(model, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    max_iterations=3,
)

agent_executor.invoke({"input": query})



> Entering new AgentExecutor chain...

Invoking: `magic_function` with `{'input': '3'}`


Sorry, there was an error. Please try again.
Invoking: `magic_function` with `{'input': '3'}`
responded: It seems there was an error when trying to retrieve the value of the magic function with input 3. Let me try again.

Sorry, there was an error. Please try again.
Invoking: `magic_function` with `{'input': '3'}`
responded: I apologize for the inconvenience. Let me try one more time to retrieve the value of the magic function with input 3.

Sorry, there was an error. Please try again.

> Finished chain.


{'input': 'what is the value of magic_function(3)?',
 'output': 'Agent stopped due to max iterations.'}

In [11]:
from langgraph.pregel import GraphRecursionError


app = chat_agent_executor.create_tool_calling_executor(model, tools)

try:
    for chunk in app.stream({"messages": [("human", query)]}, {"recursion_limit": 2*3 + 1}):
        print_chunk(chunk)
        print("------")
except GraphRecursionError:
    print(
        {"input": query, "output": "Agent stopped due to max iterations."}
    )

Invoking: [{'name': 'magic_function', 'args': {'input': '3'}, 'id': 'call_2UQudk3Ahm7XHPiMrWntAwLd'}]
------
Sorry, there was an error. Please try again.
------
Invoking: [{'name': 'magic_function', 'args': {'input': '3'}, 'id': 'call_CJ0EdqGDSyQBIP15gL5vYHEW'}]
------
Sorry, there was an error. Please try again.
------
Invoking: [{'name': 'magic_function', 'args': {'input': '3'}, 'id': 'call_ZiVN3nFEHDHEfCEh7O3w2eBO'}]
------
Sorry, there was an error. Please try again.
------
{'input': 'what is the value of magic_function(3)?', 'output': 'Agent stopped due to max iterations.'}


### `max_execution_time`

`AgentExecutor` implements a `max_execution_time` parameter, allowing users to abort a run that exceeds a total time limit.

In [12]:
import time

@tool
def magic_function(input: str) -> str:
    """Applies a magic function to an input."""
    time.sleep(2.5)
    return "Sorry, there was an error. Please try again."


tools = [magic_function]

In [13]:
agent = create_tool_calling_agent(model, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    max_execution_time=2,
    verbose=True,
)

agent_executor.invoke({"input": query})



> Entering new AgentExecutor chain...

Invoking: `magic_function` with `{'input': '3'}`


Sorry, there was an error. Please try again.

> Finished chain.


{'input': 'what is the value of magic_function(3)?',
 'output': 'Agent stopped due to max iterations.'}

LangGraph does not support constraints on the total execution time, but you can implement constraints on the duration of any individual step using `step_timeout`.

In [14]:
app = chat_agent_executor.create_tool_calling_executor(model, tools)
app.step_timeout = 2

try:
    for chunk in app.stream({"messages": [("human", query)]}):
        print_chunk(chunk)
        print("------")
except TimeoutError:
    print(
        {"input": query, "output": "Agent stopped due to max iterations."}
    )

Invoking: [{'name': 'magic_function', 'args': {'input': '3'}, 'id': 'call_PWHVgXhbM3P80uqw9WsG3T9P'}]
------
{'input': 'what is the value of magic_function(3)?', 'output': 'Agent stopped due to max iterations.'}


## 3. Handling errors

Note: pending https://github.com/langchain-ai/langgraph/pull/319

When using tool-calling models, both LangGraph's built-in `create_tool_calling_executor` and AgentExecutor support handling of parsing errors.

Below, we construct a model whose first response includes a malformed tool call.

In [15]:
from typing import List

from langchain_core.messages import AIMessage, AIMessageChunk, BaseMessage
from langchain_core.outputs import ChatGeneration, ChatGenerationChunk, ChatResult


@tool
def magic_function(input: int) -> int:
    """Applies a magic function to an input."""
    return input + 2


tools = [magic_function]


class ErrorProneModel(ChatOpenAI):

    first_message = True

    message = AIMessageChunk(
        content="",
        additional_kwargs={
            "tool_calls": [
                {
                    "id": "abc123",
                    "function": {
                        "arguments": "oops",  # <-- malformed JSON
                        "name": "magic_function",
                    },
                    "type": "function",
                },
            ],
        },
    )

    def _stream(
        self,
        messages: List[BaseMessage],
        **kwargs
    ) -> List[ChatGenerationChunk]:
        if self.first_message:
            self.first_message = False
            return [ChatGenerationChunk(message=self.message)]
        else:
            return super()._stream(messages, **kwargs)

    def _generate(self, messages: List[BaseMessage], **kwargs) -> ChatResult:
        if self.first_message:
            self.first_message = False
            return ChatResult(generations=[ChatGeneration(message=self.message)])
        else:
            return super()._generate(messages, **kwargs)

In [16]:
agent = create_tool_calling_agent(ErrorProneModel(), tools, prompt)

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    handle_parsing_errors=True,
)

agent_executor.invoke({"input": query})

{'input': 'what is the value of magic_function(3)?',
 'output': 'The value of magic_function(3) is 5.'}

AgentExecutor will retry upon parsing failures if `handle_parsing_errors` is set. See [Langsmith trace](https://smith.langchain.com/public/6f2bc739-16f4-4a52-80b6-772b3439cece/r) for the above run.

In LangGraph, we can pass in an arbitrary function that, given the message with malformed tool calls, appends new messages
for the agent to review.

In [17]:
from langchain_core.messages import HumanMessage, ToolMessage


def handle_error_function(offending_message: AIMessage) -> list:
    tool_call_id = offending_message.additional_kwargs["tool_calls"][0]["id"]
    return [
        ToolMessage(content="error", tool_call_id=tool_call_id),
        HumanMessage(content="There was an error, please try the tool again."),
    ]


app = chat_agent_executor.create_tool_calling_executor(
    ErrorProneModel(),
    tools,
    handle_parsing_errors=handle_error_function,
)


messages = app.invoke({"messages": [("human", query)]})
{
    "input": query,
    "output": messages["messages"][-1].content,
}

{'input': 'what is the value of magic_function(3)?',
 'output': 'The value of `magic_function(3)` is 5.'}

See [Langsmith trace](https://smith.langchain.com/public/3986680c-17dc-48a3-ba32-eac7e6745b38/r) for the above run.